# Mask R-CNN - Training Model

<i> Sébastien Ohleyer </i>

Training notebook.

Python 3

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import utils
import visualize
from visualize import display_images
import model as modellib
from model import log

%matplotlib inline
ROOT_DIR = os.getcwd()

/Users/sebastienohleyer/anaconda3/envs/objreco_namr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Configuration

In [2]:
import aerial
config = aerial.AerialConfig()

# Local
#AERIAL_DIR = "~/Document/ENS MVA/Object recognition/AerialImageDataset/"  # TODO: enter value here
#COCO_MODEL_PATH = "~/Document/ENS MVA/Object recognition/Mask_RCNN-coco/coco_weigths/mask_rcnn_coco.h5"
#MODEL_DIR = "../trained_model/"

# Floydhub
#AERIAL_DIR = "/"  # TODO: enter value here
#COCO_MODEL_PATH = "/coco_weights/mask_rcnn_coco.h5"
#MODEL_DIR = "/output/trained_model/"
# run : floyd run --data sohleyer/datasets/aerialimagedataset_train/1:/train --data sohleyer/datasets/coco_weights/1:/coco_weights --env tensorflow-1.3 --mode jupyter

TOWN_LIST = ["austin", "chicago", "kitsap", "tyrol-w", "vienna"]
IMAGE_PER_TOWN = 1

config.display()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (1024, 1024)
NAME                           aerial
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_T

In [2]:
# Load dataset train
dataset_train = aerial.AerialDataset()
dataset_train.load_aerial(dataset_dir=AERIAL_DIR, subset="train", town_list=TOWN_LIST, image_per_town=IMAGE_PER_TOWN)
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Info: {}".format(dataset_train.class_info))

NameError: name 'AERIAL_DIR' is not defined

In [4]:
[dataset_train.image_info[i]["image_name"] for i in dataset_train.image_ids]

['austin1.tif', 'chicago1.tif', 'kitsap1.tif', 'tyrol-w1.tif', 'vienna1.tif']

In [5]:
# Load dataset train
dataset_val = aerial.AerialDataset()
dataset_val.load_aerial(dataset_dir=AERIAL_DIR, subset="val", town_list=TOWN_LIST, image_per_town=IMAGE_PER_TOWN)
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Info: {}".format(dataset_val.class_info))

Image Count: 5
Class Info: [{'source': '', 'id': 0, 'name': 'BG'}, {'source': 'aerial', 'id': 1, 'name': 'building'}]


In [6]:
[dataset_val.image_info[i]["image_name"] for i in dataset_val.image_ids]

['austin6.tif', 'chicago6.tif', 'kitsap6.tif', 'tyrol-w6.tif', 'vienna6.tif']

## Create model

In [7]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [8]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

## Training

In [ ]:
# Stage 1 : Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs=1, layers='heads')

In [ ]:
# Stage 2
# Finetune layers from ResNet stage 4 and up
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs=20, layers='4+')

In [ ]:
# Stage 3 : Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE / 10, epochs=30, layers="all")